In [1]:
import pandas as pd
import numpy as np
from PIL import Image

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision import transforms

In [2]:
#PATH_TABULAR_DATA = "./drive/MyDrive/real_estate_price_prediction_processed_09042024.csv"
PATH_TABULAR_DATA = "./real_estate_price_prediction_processed_09042024.csv"


In [3]:
data_tab = pd.read_csv(PATH_TABULAR_DATA)

In [4]:
data_tab[["price_scaled"]] = MinMaxScaler().fit_transform(data_tab[["price"]])

In [5]:
data_tab.head()

,id_annonce,property_type,approximate_latitude,approximate_longitude,city,postal_code,size,floor,land_size,energy_performance_value,...,nb_photos,has_a_balcony,nb_terraces,has_a_cellar,has_a_garage,has_air_conditioning,last_floor,upper_floors,price,price_scaled
0,35996577,0,43.643880,7.117183,8452,199,63.0,3.479524,3995.665362,205.385148,...,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,355000.0,0.145320
1,35811033,0,45.695757,4.895610,8234,3347,90.0,3.000000,3995.665362,223.000000,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,190000.0,0.072778
2,35731841,12,47.966791,-1.220451,5235,1650,61.0,3.479524,370.000000,205.385148,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39000.0,0.006390
3,35886765,12,47.289292,-1.878805,1929,2050,142.0,3.479524,764.000000,217.000000,...,8.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,299000.0,0.120699
4,35781137,0,45.718992,4.844234,4476,3334,88.0,3.000000,3995.665362,205.385148,...,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,478000.0,0.199397


In [6]:
X_train_, X_test, y_train_, y_test= train_test_split(
    data_tab.drop(columns=["price", "price_scaled"]),
    data_tab["price_scaled"],
    test_size=0.20,
    random_state=123,
    shuffle=True
    )

In [7]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train_,
    y_train_,
    test_size=0.20,
    random_state=123,
    shuffle=True
    )

In [8]:
X_reorganized = pd.concat(
    [
        pd.DataFrame(X_train).assign(group="train"),
        pd.DataFrame(X_val).assign(group="val"),
        pd.DataFrame(X_test).assign(group="test")
    ]
).reset_index(drop=True)


y_reorganized = pd.concat(
    [
        pd.DataFrame(y_train).assign(group="train"),
        pd.DataFrame(y_val).assign(group="val"),
        pd.DataFrame(y_test).assign(group="test")
    ]
).reset_index(drop=True)

In [9]:
X_reorganized.sample(5)

,id_annonce,property_type,approximate_latitude,approximate_longitude,city,postal_code,size,floor,land_size,energy_performance_value,...,nb_boxes,nb_photos,has_a_balcony,nb_terraces,has_a_cellar,has_a_garage,has_air_conditioning,last_floor,upper_floors,group
32225,36063343,0,48.782828,2.274918,4049,4560,98.0,4.000000,3995.665362,205.385148,...,0.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,test
31299,35732195,0,48.107355,-1.683214,6174,1648,90.0,1.000000,3995.665362,132.000000,...,1.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test
17344,35778071,12,45.930519,-0.987044,6251,696,158.0,3.479524,202.000000,64.000000,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
22069,35795059,12,48.918635,2.555357,4320,4582,72.0,3.479524,270.000000,296.000000,...,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
8165,36000815,12,48.461186,1.483290,4243,1213,15115.0,3.479524,802.000000,244.000000,...,0.0,9.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,train


In [10]:
#X_reorganized[["id_annonce", "group"]]

In [11]:
#PATH_IMAGES_DATA = "./drive/MyDrive/real_estate_price_prediction_metadata_images_09042024.csv"
PATH_IMAGES_DATA = "./real_estate_price_prediction_metadata_images_09042024.csv"

In [12]:
data_img = pd.read_csv(PATH_IMAGES_DATA)

In [13]:
data_img_agg = data_img.groupby('id_annonce')['long_path_image'].apply(list).reset_index(name='images')

In [14]:
data_img_groups = data_img_agg.merge(
    X_reorganized[["id_annonce", "group"]],
    how='left',
    on='id_annonce'
)

In [15]:
data_img_groups.sample(10)

,id_annonce,images,group
737,32014329,[C:\Users\MI30743\github\bourbaki_challenges-m...,train
1255,35027961,[C:\Users\MI30743\github\bourbaki_challenges-m...,train
12257,35830207,[C:\Users\MI30743\github\bourbaki_challenges-m...,train
4493,35767987,[C:\Users\MI30743\github\bourbaki_challenges-m...,train
27303,36003531,[C:\Users\MI30743\github\bourbaki_challenges-m...,train
32712,36041965,[C:\Users\MI30743\github\bourbaki_challenges-m...,train
2381,35745733,[C:\Users\MI30743\github\bourbaki_challenges-m...,val
3022,35754883,[C:\Users\MI30743\github\bourbaki_challenges-m...,test
3970,35762953,[C:\Users\MI30743\github\bourbaki_challenges-m...,val
22112,35897555,[C:\Users\MI30743\github\bourbaki_challenges-m...,val


In [16]:
# Define the image transformation
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
        ),
])


In [17]:
mapeo = {}
model_ = models.resnet152(pretrained=True)
model_layers = list(model_.children())[:-1]
model_feature_extract = torch.nn.Sequential(*model_layers)

def get_model_embedding(model_feature_extract, img_tensor):
    with torch.no_grad():
          # Get the image features from the ResNet-152 model
          img_features = model_feature_extract(img_tensor)
    return img_features

# for index in range(data_img_groups.shape[0]):

#   images_path = data_img_groups.iloc[index]["images"]

#   images_emmbeded = []

#   for path in images_path:
#     image = Image.open(path).convert("RGB")

#     image = preprocess(image).unsqueeze(0)

#     img_embedding = get_model_embedding(model_feature_extract, image)
#     images_emmbeded.append(img_embedding.flatten())

#   images_embbeding = torch.stack(images_emmbeded).mean(dim=0)

#   mapeo[index] = images_embbeding


c:\Users\MI30743\github\bourbaki_challenges-main\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\MI30743\github\bourbaki_challenges-main\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [29]:
data_img["long_path_image"]

0         C:\Users\MI30743\github\bourbaki_challenges-ma...
1         C:\Users\MI30743\github\bourbaki_challenges-ma...
2         C:\Users\MI30743\github\bourbaki_challenges-ma...
3         C:\Users\MI30743\github\bourbaki_challenges-ma...
4         C:\Users\MI30743\github\bourbaki_challenges-ma...
                                ...                        
190694    C:\Users\MI30743\github\bourbaki_challenges-ma...
190695    C:\Users\MI30743\github\bourbaki_challenges-ma...
190696    C:\Users\MI30743\github\bourbaki_challenges-ma...
190697    C:\Users\MI30743\github\bourbaki_challenges-ma...
190698    C:\Users\MI30743\github\bourbaki_challenges-ma...
Name: long_path_image, Length: 190699, dtype: object

In [32]:
import multiprocessing


def get_raw_embbeding(index):
    def get_model_embedding(model_feature_extract, img_tensor):
        with torch.no_grad():
            # Get the image features from the ResNet-152 model
            img_features = model_feature_extract(img_tensor)
        return img_features

    path = data_img.iloc[index]["long_path_image"]

    images_emmbeded = []

    image = Image.open(path).convert("RGB")

    image = preprocess(image).unsqueeze(0)

    img_embedding = get_model_embedding(model_feature_extract, image)

    return img_embedding.flatten()

def get_avg_embbeding(index):
    def get_model_embedding(model_feature_extract, img_tensor):
        with torch.no_grad():
            # Get the image features from the ResNet-152 model
            img_features = model_feature_extract(img_tensor)
        return img_features

    images_path = data_img_groups.iloc[index]["images"]

    images_emmbeded = []

    for path in images_path:
        image = Image.open(path).convert("RGB")

        image = preprocess(image).unsqueeze(0)

        img_embedding = get_model_embedding(model_feature_extract, image)
        images_emmbeded.append(img_embedding.flatten())

    images_embbeding = torch.stack(images_emmbeded).mean(dim=0)

    return images_embbeding

In [39]:
data_img.shape

(190699, 5)

In [46]:
13*190699/1000/60/24

1.7215881944444444

In [40]:
%%time
[get_raw_embbeding(i) for i in range(1000)]

CPU times: total: 13min 15s
Wall time: 2min 45s


[tensor([0.7384, 0.5574, 0.8128,  ..., 0.3975, 0.3546, 0.4288]),
 tensor([0.8412, 0.5168, 0.7020,  ..., 0.4043, 0.3513, 0.4753]),
 tensor([0.8854, 0.5093, 0.8406,  ..., 0.3945, 0.3290, 0.3890]),
 tensor([0.7911, 0.5185, 0.8045,  ..., 0.4095, 0.3848, 0.4313]),
 tensor([0.8222, 0.4794, 0.9182,  ..., 0.4049, 0.3069, 0.4459]),
 tensor([0.7644, 0.4993, 0.7606,  ..., 0.3951, 0.3528, 0.4032]),
 tensor([0.7193, 0.5441, 0.7415,  ..., 0.4360, 0.3712, 0.3972]),
 tensor([0.7699, 0.4269, 0.8118,  ..., 0.3803, 0.3526, 0.4502]),
 tensor([0.8072, 0.4656, 0.7619,  ..., 0.4199, 0.3423, 0.3952]),
 tensor([0.7506, 0.4525, 0.7262,  ..., 0.4382, 0.3405, 0.4412]),
 tensor([0.7719, 0.5273, 0.7688,  ..., 0.3575, 0.3992, 0.3930]),
 tensor([0.8855, 0.5147, 0.7012,  ..., 0.3866, 0.3729, 0.4118]),
 tensor([0.7477, 0.4956, 0.7482,  ..., 0.4345, 0.3961, 0.4027]),
 tensor([0.7860, 0.4843, 0.7584,  ..., 0.3802, 0.4029, 0.4643]),
 tensor([0.7552, 0.4747, 0.7119,  ..., 0.3784, 0.3727, 0.3562]),
 tensor([0.7948, 0.4939, 

In [22]:
data_img_groups.shape

(37368, 3)

In [19]:
pool_size = multiprocessing.cpu_count()

pool_size

20

In [23]:
%%time 
results = [get_avg_embbeding(i) for i in range(100)]


CPU times: total: 6min 59s
Wall time: 1min 24s


In [22]:
get_avg_embbeding(2)

tensor([0.8398, 0.4764, 0.7844,  ..., 0.4107, 0.3604, 0.4081])

In [23]:
%%time
def parallarel_procesing(numbers, pool_size):
    pool = multiprocessing.Pool(pool_size)
    results = pool.map(get_avg_embbeding, numbers)

    return results

index_images = range(2)#range(data_img_groups.shape[0])

results = parallarel_procesing(index_images, pool_size)


In [63]:
data_img_groups.shape[0]

37368

In [ ]:
import json 

mapeo_ = dict(zip(list(data_img_groups["id_annonce"].values), [ list(x.numpy()) for x in results]))

with open("average_embbedings_images.json", "w") as outfile: 
    json.dump(mapeo_, outfile)

In [17]:
class CustomDataset(Dataset):

    def __init__(
        self,
        data_features,
        data_target,
        data_img_groups,
        group='train',
        transform=None
        ) -> None:

        self.group = group

        self.data_features = data_features.query(
            f"group == '{self.group}'"
            )

        self.data_target = data_target.query(
            f"group == '{self.group}'"
            )

        self.data_img_groups = data_img_groups.query(
            f"group == '{self.group}'"
            )

        self.transform = transform

        self.model = models.resnet152(pretrained=True)
        self.model_layers = list(self.model.children())[:-1]
        self.model_feature_extract = torch.nn.Sequential(*self.model_layers)

        self.set_model_eval()

    def __len__(self):
        return len(self.data_features)

    def set_model_eval(self):
      """
      Set to evaluation model.
      """
      return self.model.eval()


    def get_model_embedding(self, img_tensor):

      with torch.no_grad():
          # Get the image features from the ResNet-152 model
          img_features = self.model_feature_extract(img_tensor)

      return img_features

    def load_image(self, path) -> Image.Image:

        return Image.open(path).convert("RGB")

    def __getitem__(self, index):

        images_path = self.data_img_groups.iloc[index]["images"]

        images_emmbeded = []

        for path in images_path:
          image = self.load_image(path)

          if self.transform:
            image = self.transform(image).unsqueeze(0)

            img_embedding = self.get_model_embedding(image)
            images_emmbeded.append(img_embedding.flatten())

        images_embbeding = torch.stack(images_emmbeded).mean(dim=0)

        features = self.data_features.drop(
            columns=["id_annonce","group"]
            ).iloc[index,:]

        #features = torch.from_numpy(features).float()

        target = self.data_target[["price_scaled"]].values[index]
        #target = np.array(target)
        #target = torch.from_numpy(target).float()

        #sample = {}

        avg_embbeding = images_embbeding
        tab_features = torch.tensor(features.values, dtype=torch.float32)
        target = torch.tensor(target, dtype=torch.float32)

        return avg_embbeding, tab_features, target

In [34]:
train_dataset = CustomDataset(
        data_features=X_reorganized.head(100),
        data_target=y_reorganized.head(100),
        data_img_groups=data_img_groups,
        group='train',
        transform=preprocess
    )

val_dataset = CustomDataset(
        data_features=X_reorganized.sample(100),
        data_target=y_reorganized.sample(100),
        data_img_groups=data_img_groups,
        group='val',
        transform=preprocess
)

test_dataset = CustomDataset(
        data_features=X_reorganized.sample(100),
        data_target=y_reorganized.sample(100),
        data_img_groups=data_img_groups,
        group='test',
        transform=preprocess
)

c:\Users\MI30743\github\bourbaki_challenges-main\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\MI30743\github\bourbaki_challenges-main\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\MI30743\github\bourbaki_challenges-main\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\MI30743\github\bourbaki_challenges-main\.venv\L

In [35]:
BATCH_SIZE = 20

In [36]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [37]:
# Define the neural network
class PricePredictionModel(nn.Module):
    def __init__(self, embedding_size=2048, tabular_input_size=26):  # Adjust tabular_input_size based on your actual features
        super(PricePredictionModel, self).__init__()
        self.fc1 = nn.Linear(embedding_size+tabular_input_size, 512)  # Output a single value for price prediction
        self.fc2 = nn.Linear(512, 1)

    def forward(self, image_inputs, tabular_inputs):
        combined_embedding = torch.cat((image_inputs, tabular_inputs), dim=1)
        output = self.fc1(combined_embedding)
        output = self.fc2(output)
        return output.squeeze(1)

In [22]:
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"

In [23]:
device

'cuda:0'

In [24]:
#model = MultiModal(device=device)

In [25]:
# Initialize the model, loss, and optimizer
model = PricePredictionModel().to(device)

In [26]:
class EarlyStopping:
    def __init__(self, patience, verbose=False, delta=0, path="checkpoint.pt"):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        return self.early_stop

    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decrease."""
        if self.verbose:
            print(
                f"Validation Loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model to {self.path}"
            )
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

    def load_checkpoint(self, model):
        """Loads the best model weights from the saved checkpoint."""
        model.load_state_dict(torch.load(self.path))

In [27]:
# Tunners
LR = 0.01
EPOCHS = 1

In [44]:
optimizer = optim.Adam(model.parameters(), lr=LR, amsgrad=True, weight_decay=LR * 0.1)

criterion = nn.L1Loss().to(device) #nn.L1Loss().to(device)

early_stopping = EarlyStopping(patience=int(EPOCHS * 0.5), verbose=True)

In [45]:
def train_eval(model, loader, optimizer, loss_func, is_training):
    loss_list = []
    stop = False
    model.train() if is_training else model.eval()

    with torch.set_grad_enabled(is_training):  # Enable gradients only in training
        for batch_idx, (avg_embbeding, tab_features, target) in tqdm(
            enumerate(loader),
            desc="Epoch_Train" if is_training else "Epoch_Val",
            total=len(loader),
        ):
            avg_embbeding, tab_features, target = avg_embbeding.to(device), tab_features.to(device), target.to(device)

            if is_training:
                optimizer.zero_grad()  # Reset gradients
            output = model(avg_embbeding, tab_features)
            loss = loss_func(output, target.squeeze(1))

            if is_training:
                loss.backward()  # Compute gradients
                optimizer.step()  # Update weights

            loss_list.append(loss.item())

    if not is_training:
        stop = early_stopping(
            (sum(loss_list) / len(loss_list)), model
        )  # Early stopping decision

    return loss_list, stop

In [40]:
def calc_acc_loss(total_loss, loss):
    total_loss.append(sum(loss) / len(loss))

    #total_accuracy.append(100 * correct / len(loader.dataset))
    return total_loss

In [43]:
# Lists
total_loss_train = []
total_loss_val = []

In [46]:
torch.cuda.empty_cache()
try:
  torch.cuda.reset_peak_memory_stats()
except:
  pass

In [47]:
%%time
# Training and Evaluation loop
for epoch in range(EPOCHS):
    # Train Iteration
    loss_train, _ = train_eval(
        model, train_loader, optimizer, criterion, is_training=True
    )

    # Val Iteration
    loss_val, stop = train_eval(
        model, val_loader, optimizer, criterion, is_training=False
    )

    # Calculate and record loss & accuracy for training and validation
    total_loss_train = calc_acc_loss(
        total_loss_train,
        loss_train
    )

    # Calculate Loss and Accuracy
    total_loss_val = calc_acc_loss(
        total_loss_val,
        loss_val
    )

    # Print metrics per epoch
    print(
        f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {total_loss_train[-1]:.4f}, Val Loss: {total_loss_val[-1]:.4f}"
    )

    if stop:
        print("Early Stopping")
        break

Epoch_Train:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch_Val:   0%|          | 0/1 [00:00<?, ?it/s]

Validation Loss decreased (inf --> 3939.195801). Saving model to checkpoint.pt
Epoch 1/1 - Train Loss: 710.4358, Val Loss: 3939.1958
CPU times: total: 6min 21s
Wall time: 1min 20s


## Texting

In [48]:
try:
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()
except:
  pass

In [49]:
model_pred = PricePredictionModel().to(device) #MultiModal(device=device)
model_pred.load_state_dict(torch.load("checkpoint.pt"))
model_pred.to(device)

PricePredictionModel(
  (fc1): Linear(in_features=2074, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=1, bias=True)
)

In [56]:
def test(model, loader):
    model.eval()
    total_loss_test = []
    outputs_test = []
    target_test = []

    pred = torch.Tensor().to(device)

    with torch.no_grad():
        for batch_idx, (avg_embbeding, tab_features, target) in tqdm(
            enumerate(loader), desc="Test Epoch", total=len(loader)
        ):
            avg_embbeding, tab_features, target = avg_embbeding.to(device), tab_features.to(device), target.to(device)
            output = model(avg_embbeding, tab_features)

            outputs_test.append(output)
            target_test.append(target)

            loss = criterion(output, target.squeeze(1))
            total_loss_test.append(loss.item())

        outputs_test = torch.concat(outputs_test, dim=0).cpu().numpy()
        target_test = torch.concat(target_test, dim=0).cpu().numpy()

        return total_loss_test, outputs_test, target_test.squeeze()

In [57]:
total_loss_test, outputs_test, target_test = test(model_pred, test_loader)

Test Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

In [58]:
print(f"Loss Test: {(sum(total_loss_test) / len(total_loss_test)):.3f}")

Loss Test: 929.151


In [59]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(outputs_test, target_test)

929.1509